<a href="https://colab.research.google.com/github/Madhu3456/IITISoC-22-AI-ML-11-Song-Recommendation/blob/main/FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import keras
import os
import sys
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import warnings
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LeakyReLU
import cv2


Read data from G-drive

In [6]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fer2013.csv")
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


To check if the dataset is empty

In [7]:
df.isnull().sum()

emotion    0
pixels     0
Usage      0
dtype: int64

*Data PreProcessing*

Shuffle data

In [8]:
df = df.sample(frac=1)

Changing image pixels to Numpy array

In [9]:
train_pixels = df["pixels"].astype(str).str.split(" ").tolist()
train_pixels = np.uint8(train_pixels)

Standardization

In [10]:
pixels = train_pixels.reshape((35887*2304,1))

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pixels = scaler.fit_transform(pixels)

In [12]:
pixels = train_pixels.reshape((35887, 48, 48,1))

In [9]:
df['pixels']

15206    248 143 113 85 53 65 98 128 157 180 189 195 20...
8691     64 61 55 34 54 64 50 46 47 32 43 47 45 40 39 4...
2391     104 110 90 78 72 84 103 126 144 153 159 160 16...
8567     250 249 245 249 250 249 249 250 255 255 255 25...
35400    197 49 53 53 66 78 82 81 83 85 88 89 91 88 103...
                               ...                        
24241    16 11 14 9 9 6 7 7 12 19 22 20 19 24 23 21 22 ...
20916    67 89 126 159 176 184 186 190 195 199 204 209 ...
7697     72 122 122 103 99 137 142 150 159 152 169 154 ...
25912    97 102 100 97 98 102 98 102 99 42 31 28 40 45 ...
8794     255 255 255 239 221 218 195 196 180 178 191 20...
Name: pixels, Length: 35887, dtype: object

Illumination Normalisation

In [13]:
def normalize(img):
  lmin = float(img.min())
  lmax = float(img.max())
  return np.floor((img-lmin)/(lmax-lmin*255.0))

  for i in range(0,35887):
    df['pixels'][i] = normalize(df['pixels'][i] )

Test Train data split

One Hot Encoding

In [14]:
labels = to_categorical(df['emotion'], num_classes=7)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(pixels,labels, test_size=0.2,shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,shuffle=False)

Data Augumentation

In [16]:
datagen = ImageDataGenerator(  width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2)
valgen = ImageDataGenerator(   width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2) 

In [17]:
datagen.fit(X_train)
valgen.fit(X_val)

train_generator = datagen.flow(X_train, y_train, batch_size=64)
val_generator = datagen.flow(X_val, y_val, batch_size=64)



Model

In [24]:
width,height = 48,48

In [41]:

network = Sequential()

network.add(Conv2D(32, (3,3), activation='relu', padding = 'same', input_shape = (width, height, 1)))
network.add(BatchNormalization())
network.add(Conv2D(64, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(128, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(128, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(256, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(256, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(512, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(512, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Flatten())

network.add(Dense(256, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(64, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(7, activation='softmax'))

Early stopping to prevent overfitting

In [42]:
optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.99,decay=0.001/32)
EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
callback=[EarlyStop , Reduce_LR]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


compile model

In [19]:
network.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=["accuracy"])

NameError: ignored

Fit model

In [44]:
history=network.fit(train_generator,epochs=50, callbacks=callback, verbose=2)

Epoch 1/50
359/359 - 29s - loss: 1.9531 - accuracy: 0.2500 - lr: 0.0010 - 29s/epoch - 82ms/step
Epoch 2/50
359/359 - 15s - loss: 1.6004 - accuracy: 0.3901 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 3/50
359/359 - 15s - loss: 1.3751 - accuracy: 0.4721 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 4/50
359/359 - 15s - loss: 1.2747 - accuracy: 0.5147 - lr: 0.0010 - 15s/epoch - 43ms/step
Epoch 5/50
359/359 - 15s - loss: 1.2143 - accuracy: 0.5445 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 6/50
359/359 - 15s - loss: 1.1569 - accuracy: 0.5658 - lr: 0.0010 - 15s/epoch - 42ms/step
Epoch 7/50
359/359 - 15s - loss: 1.1176 - accuracy: 0.5769 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 8/50
359/359 - 15s - loss: 1.0724 - accuracy: 0.5974 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 9/50
359/359 - 15s - loss: 1.0549 - accuracy: 0.6074 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 10/50
359/359 - 15s - loss: 1.0197 - accuracy: 0.6205 - lr: 0.0010 - 15s/epoch - 41ms/step
Epoch 11/50
359/359 - 15s - loss: 1.002

In [45]:
network.save('emotions.h5')

In [46]:
model = tf.keras.models.load_model('emotions.h5')

In [47]:
loss = model.evaluate(X_test,y_test) 
print("Test Acc: " + str(loss[1]))

225/225 [==============================] - 3s 9ms/step - loss: 1.0946 - accuracy: 0.6700
Test Acc: 0.6699637770652771


Music Recomendation

In [3]:
mood_music = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_moods.csv")
mood_music = mood_music[['name','artist','mood']]
mood_music.head()

,name,artist,mood
0,1999,Prince,Happy
1,23,Blonde Redhead,Sad
2,9 Crimes,Damien Rice,Sad
3,99 Luftballons,Nena,Happy
4,A Boy Brushed Red Living In Black And White,Underoath,Energetic


In [21]:
input_file = sys.argv[1]
image = cv2.imread(input_file)

In [ ]:
image = cv2.resize(image,(48,48))
img=np.array(image)
img=img.reshape(1,48,48,1)
predict_x=network.predict(img) 
result=np.argmax(predict_x,axis=1)
result[0]

In [ ]:
if(result[0]==0 or result[0]==1 or result[0]==2 ):
    #for angery,disgust,fear
    filter1=mood_music['mood']=='Calm'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)
if(result[0]==3 or result[0]==4):
    #for happy, neutral
    filter1=mood_music['mood']=='Happy'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)
if(result[0]==5):
       #for Sad
    filter1=mood_music['mood']=='Sad'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)
if(result[0]==6):
     #for surprise
    filter1=mood_music['mood']=='Energetic'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)